In [ ]:
#!/usr/bin/env python3
# encoding: utf-8
#
# Copyright (C) 2024 Max Planck Institute for Multidisclplinary Sciences
# Copyright (C) 2024 Bharti Arora <bharti.arora@mpinat.mpg.de>
# Copyright (C) 2024 Ajinkya Kulkarni <ajinkya.kulkarni@mpinat.mpg.de>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program. If not, see <https://www.gnu.org/licenses/>.

########################################################################################

In [ ]:
import sys
# Don't generate the __pycache__ folder locally
sys.dont_write_bytecode = True
# Print exception without the buit-in python warning
sys.tracebacklimit = 0

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

import os
import glob
from tqdm.auto import tqdm

from modules_2photon import *

In [ ]:
FilterKey = 2
LocalDensityKey = 20
LocalSigmaKey = 10
kernel_size_circular_variance = 100

In [ ]:
folder_path = "Data"

# Get all .tif files from the directory using os.path.join
all_tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

# Prefixes to exclude
exclude_prefixes = ['FilteredImage_', 'DensityImage_', 'CoheranceImage_', 'OrientationImage_',
                    'CircularVarianceImage_', 'Results_']

# Filter out files starting with the specified prefixes
filenames = [f for f in all_tif_files if not any(os.path.basename(f).startswith(prefix)
                                                 for prefix in exclude_prefixes)]

filenames = sorted(filenames)

print(filenames)

In [ ]:
for uploaded_file_path in tqdm(filenames, desc = 'Analyzing files', leave = True):

    raw_image = np.array(Image.open(uploaded_file_path))

    raw_image = 255 * ((raw_image - raw_image.min()) / (raw_image.max() - raw_image.min()))

    ThresholdValueKey = max(int(np.median(raw_image)), 2)

    filtered_image = make_filtered_image(raw_image, FilterKey)

    binarized_image = binarize_image(filtered_image)

    # Define the kernel and it's size
    local_kernel_size = LocalDensityKey
    if (local_kernel_size % 2 == 0):
        local_kernel_size = local_kernel_size + 1
    if (local_kernel_size < 3):
        local_kernel_size = 3

    local_kernel = np.ones((local_kernel_size, local_kernel_size),
                           dtype = np.float32) / (local_kernel_size * local_kernel_size)

    Local_Density = convolve(raw_image, local_kernel)

    Local_Density = np.divide(Local_Density, Local_Density.max(),
                              out=np.full(Local_Density.shape, np.nan),
                              where=Local_Density.max() != 0)

    # Calculate image gradients in X and Y directions
    image_gradient_x, image_gradient_y = make_image_gradients(filtered_image)

    # Calculate the structure tensor and solve for EigenValues, EigenVectors
    Structure_Tensor, EigenValues, EigenVectors, Jxx, Jxy, Jyy = make_structure_tensor_2d(image_gradient_x,
                                                                                          image_gradient_y,
                                                                                          LocalSigmaKey)

    # Calculate Coherence
    Image_Coherance = make_coherence(filtered_image, EigenValues, Structure_Tensor, ThresholdValueKey)

    
    # Perform statistical analysis

    results_array = perform_statistical_analysis(uploaded_file, LocalSigmaKey, Image_Coherance )

    
    # Save the results to a CSV file
    csv_filename = os.path.splitext(os.path.basename(uploaded_file_path))[0] + '_results.csv'
    save_results_to_csv(results_array, csv_filename)